<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Get-Top-Predict-Substitute" data-toc-modified-id="Get-Top-Predict-Substitute-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Get Top Predict Substitute</a></span></li><li><span><a href="#get-Top--Actual" data-toc-modified-id="get-Top--Actual-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>get Top  Actual</a></span></li><li><span><a href="#compare--Top-Predict-In-Top-Actual" data-toc-modified-id="compare--Top-Predict-In-Top-Actual-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>compare  Top Predict In Top Actual</a></span><ul class="toc-item"><li><span><a href="#Zvi-recomandation" data-toc-modified-id="Zvi-recomandation-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Zvi recomandation</a></span><ul class="toc-item"><li><span><a href="#is-best-of-recomeneation-in-top-2-actual" data-toc-modified-id="is-best-of-recomeneation-in-top-2-actual-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>is best of recomeneation in top 2 actual</a></span></li><li><span><a href="#is-2-best-of-recomeneation-in-top-3-actual" data-toc-modified-id="is-2-best-of-recomeneation-in-top-3-actual-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>is 2 best of recomeneation in top 3 actual</a></span></li></ul></li></ul></li></ul></div>

# Get Top Predict Substitute

In [118]:
clf_RF = RandomForestClassifier(n_estimators=200, criterion='entropy', max_depth=None, min_samples_split=20, ccp_alpha=0.0003, random_state=1234)

In [121]:
clf_RF.fit(X_train, y_train)

RandomForestClassifier(ccp_alpha=0.0003, criterion='entropy',
                       min_samples_split=20, n_estimators=200,
                       random_state=1234)

In [299]:
def getBestSubstitute(model, X, numOfSubs=2 ):
        # get soft prediction
        y_pred_soft=model.predict_proba(X)
        df_proba=pd.concat([X.reset_index(drop=True),pd.DataFrame(y_pred_soft[:,1],columns=['prdictProba'])],axis=1)
        # get mean and std for each  product and Subs
        dfGroup=df_proba.groupby(['ParentProductNumber','ParentProductNumberSubs']).agg({'prdictProba':[('prdictProbaMean','mean'),
                                                                                       ('prdictProbaStd',np.std),
                                                                       ('countRows','count')]}).reset_index()
        dfGroup.columns=dfGroup.columns.droplevel(0)
        dfGroup.columns.values[0]='ParentProductNumber'
        dfGroup.columns.values[1]='ParentProductNumberSubs'
        dfGroup=dfGroup[dfGroup['countRows']>=3]
        # get rank of mean and std
        dfGroup['scoreMean']=dfGroup.groupby('ParentProductNumber')['prdictProbaMean'].apply(lambda x: (x-x.min())/(x.max()-x.min()))
        dfGroup['scoreStd']=dfGroup.groupby('ParentProductNumber')['prdictProbaStd'].apply(lambda x: 1-((x-x.min())/(x.max()-x.min())))
        dfGroup['scoreMeanAndStd']=(dfGroup['scoreMean']*0.8+dfGroup['scoreStd']*0.2).astype(np.float64)
        dfGroup=dfGroup.round({'scoreMeanAndStd':4})
        dfGroup['rankMeanAndStd']=dfGroup.groupby('ParentProductNumber')['scoreMeanAndStd'].rank(ascending=0,method='dense')
        dfGroup['rankMeanAndStd']=np.where(dfGroup['rankMeanAndStd'].isnull(),1,dfGroup['rankMeanAndStd'])
        dfGroup.sort_values(['ParentProductNumber','rankMeanAndStd'],inplace=True)
        # filter best susbs by numofSubs
        dfGroupFinal=dfGroup[(dfGroup['rankMeanAndStd']<=numOfSubs)]
        return dfGroupFinal

In [300]:
df_bestTwoSubs=getBestSubstitute(clf_RF, X)

In [301]:
df_bestTwoSubs.head()

,ParentProductNumber,ParentProductNumberSubs,prdictProbaMean,prdictProbaStd,countRows,scoreMean,scoreStd,scoreMeanAndStd,rankMeanAndStd
12,161631,7290112497918,0.91,0.12,24,1.00,1.00,1.00,1.0
0,161631,161648,0.89,0.17,45,0.94,0.82,0.92,2.0
30,161648,7290112497918,0.91,0.12,31,1.00,1.00,1.00,1.0
16,161648,72909799,0.90,0.15,68,0.60,0.39,0.55,2.0
41,170053,7290104720536,0.78,0.08,11,1.00,0.86,0.97,1.0


In [302]:
df_bestTwoSubs.shape

(935, 9)

# get Top  Actual

In [340]:
def getTopActual(df,NTop=2):
    df_groupActual=df.groupby(['ParentProductNumber','ParentProductNumberSubs']).agg({'CorrelateItemStatusCode':[('AvgApproval','mean'),
                                                                                              ('StdApproval',np.std),
                                                                                              ('cntRows','count')
                                                                                             ]}).reset_index()
    df_groupActual.columns=df_groupActual.columns.droplevel(0)
    df_groupActual.columns.values[0]='ParentProductNumber'
    df_groupActual.columns.values[1]='ParentProductNumberSubs'
    df_groupActual=df_groupActual[df_groupActual['cntRows']>=3]
    df_groupActual.sort_values(['ParentProductNumber','AvgApproval'],ascending=[True,False],inplace=True)
    df_ActualTopN=df_groupActual.groupby('ParentProductNumber').head(NTop)
    return df_ActualTopN

In [341]:
df_ActualTop3=getTopActual(df,3)

In [373]:
df_ActualTop3.head(8)

,ParentProductNumber,ParentProductNumberSubs,AvgApproval,StdApproval,cntRows
12,161631,7290112497918,0.92,0.28,24
0,161631,161648,0.91,0.29,45
1,161631,72909799,0.88,0.32,86
16,161648,72909799,0.94,0.24,68
30,161648,7290112497918,0.94,0.25,31
14,161648,161631,0.92,0.28,71
41,170053,7290104720536,0.91,0.30,11
35,170053,5197628,0.80,0.41,15


# compare  Top Predict In Top Actual

In [416]:
def precKPredictInNActual(KToppredict=1, NTopActual=2, isZvi=0):
    df_ActualTop=getTopActual(df,NTopActual)
    display(df_ActualTop.head(3))
    if isZvi:
        print('Zvi rec')
        df_RecTop=ZviRec[ZviRec['RankSubs']<=KToppredict].copy()
    else:
        df_RecTop=getBestSubstitute(clf_RF, X,KToppredict)
    df_isInPred=df_ActualTop.merge(df_RecTop,on=['ParentProductNumber','ParentProductNumberSubs'],how='left')\
    .groupby('ParentProductNumber').agg({'prdictProbaMean':[('countSubs','count')],
                                        'ParentProductNumberSubs':[('countRowsMainPrd','count')]}).reset_index()
    df_isInPred.columns=df_isInPred.columns.droplevel(0)
    df_isInPred.columns.values[0]='ParentProductNumber'
    display(df_isInPred[df_isInPred['ParentProductNumber']==161631])
    
    print(len(df_isInPred[((df_isInPred['countSubs']<KToppredict) & (df_isInPred['countRowsMainPrd']>KToppredict))|
          (df_isInPred['countSubs']==0)]))
    print(len(df_isInPred[(df_isInPred['countSubs']>=KToppredict) |
          ((df_isInPred['countRowsMainPrd']<=KToppredict) &(df_isInPred['countSubs']>0))]))
    print("{:.0%}".format(len(df_isInPred[(df_isInPred['countSubs']>=KToppredict) |
          ((df_isInPred['countRowsMainPrd']<=KToppredict) &(df_isInPred['countSubs']>0))])/len(df_isInPred)))
    return df_isInPred

In [370]:
precKPredictInNActual()

,ParentProductNumber,ParentProductNumberSubs,AvgApproval,StdApproval,cntRows
12,161631,7290112497918,0.92,0.28,24
0,161631,161648,0.91,0.29,45
16,161648,72909799,0.94,0.24,68


,ParentProductNumber,countSubs,countRowsMainPrd
0,161631,1,2


56
449
89%


In [371]:
precKPredictInNActual(2,3)

,ParentProductNumber,ParentProductNumberSubs,AvgApproval,StdApproval,cntRows
12,161631,7290112497918,0.92,0.28,24
0,161631,161648,0.91,0.29,45
1,161631,72909799,0.88,0.32,86


,ParentProductNumber,countSubs,countRowsMainPrd
0,161631,2,3


76
429
85%


## Zvi recomandation

In [406]:
ZviRec=pd.read_csv('zvi_recommendation_list (1).csv')

In [407]:
ZviRec['RankSubs']=ZviRec.groupby('Product')['AvgCorrect'].rank(ascending=0,method='dense')

In [408]:
ZviRec.head()

,Product,ProductSubs,CountCorrect,AvgCorrect,RankSubs
0,161631,161648,45,0.93,1.0
1,161631,7290112497918,24,0.92,2.0
2,161631,72909799,86,0.88,3.0
3,161648,161631,71,0.99,1.0
4,161648,72909799,68,0.94,2.0


In [409]:
ZviRec.shape

(1283, 5)

In [410]:
ZviRec.rename(columns={'Product':'ParentProductNumber',
                      'ProductSubs':'ParentProductNumberSubs',
                       'AvgCorrect':'prdictProbaMean'
                      },inplace=True)

### is best of recomeneation in top 2 actual

In [417]:
precKPredictInNActual(1,2,1)

,ParentProductNumber,ParentProductNumberSubs,AvgApproval,StdApproval,cntRows
12,161631,7290112497918,0.92,0.28,24
0,161631,161648,0.91,0.29,45
16,161648,72909799,0.94,0.24,68


Zvi rec


,ParentProductNumber,countSubs,countRowsMainPrd
0,161631,1,2


121
384
76%


### is 2 best of recomeneation in top 3 actual

In [415]:
precKPredictInNActual(2,3,1)

,ParentProductNumber,ParentProductNumberSubs,AvgApproval,StdApproval,cntRows
12,161631,7290112497918,0.92,0.28,24
0,161631,161648,0.91,0.29,45
1,161631,72909799,0.88,0.32,86


Zvi rec


,ParentProductNumber,countSubs,countRowsMainPrd
0,161631,2,3


125
380
75%
